# Concorrenza e Parallelismo

import necessari

In [5]:
import time
import threading
import hashlib


# import sys
# sys.setswitchinterval(0.5)

### Effetti del GIL sui thread

Esercizio 1 - conto alla rovescia

In [ ]:
def conto_alla_rovescia(n):
    while n > 0:
        n -= 1

In [6]:
N = 100_000_000
LISTA_NUM_THREAD = [1, 2, 4, 8]

In [11]:
def esegui_in_thread(num_thread, func, *args):
    threads = [threading.Thread(target=func, args=args) for _ in range(num_thread)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

In [12]:
def calcola_tempi():
    for num_thread in LISTA_NUM_THREAD:
        n = N //num_thread
        print(f'{num_thread=}; {n=}')
        for _ in range(3):
            inizio = time.time()
            esegui_in_thread(num_thread, conto_alla_rovescia, n)
            print(time.time() - inizio)

In [13]:
calcola_tempi()

num_thread=1; n=100000000
2.669037103652954
2.590924024581909
2.5581510066986084
num_thread=2; n=50000000
2.623228073120117
2.617216110229492
2.6049060821533203
num_thread=4; n=25000000
2.6123921871185303
2.595703125
2.7128028869628906
num_thread=8; n=12500000
2.629438877105713
2.744590997695923
2.6475181579589844


Esercizio 2 - hashlib

In [14]:
# SHA-256 hash di 8 messaggi da 128 MB
def funzione_hash(messaggio):
    return [hashlib.sha256(m).digest() for m in messaggio]


def crea_threads(n, funzione):
    threads = []
    arg = [b'0' * ((2 ** 30) // 8) for _ in range(8 // n)]      # 2^30 bytes ('0' è 1 byte) = 1 GB
    for i in range(n):
        t = threading.Thread(target=funzione, args=(arg,))
        threads.append(t)

    start = time.perf_counter()
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    end = time.perf_counter() - start
    # print(f"Completato con {n} threads in {round(end, 5)} secondi")
    return end


def esegui_media(num_media, num_threads, funzione):
    media = 0.0
    for i in range(num_media):
        media += crea_threads(num_threads, funzione)
    media /= num_media
    print(f"Media: {round(media, 5)} secondi - {num_threads} threads")

In [15]:
print("Inizio test")
inizio = time.perf_counter()

esegui_media(num_media=10, num_threads=1, funzione=funzione_hash)
esegui_media(num_media=10, num_threads=2, funzione=funzione_hash)
esegui_media(num_media=10, num_threads=4, funzione=funzione_hash)
esegui_media(num_media=10, num_threads=8, funzione=funzione_hash)

print(f"Tempo totale: {round(time.perf_counter() - inizio, 5)} secondi")

Inizio test
Media: 0.5599 secondi - 1 threads
Media: 0.23859 secondi - 2 threads
Media: 0.12699 secondi - 4 threads
Media: 0.08187 secondi - 8 threads
Tempo totale: 12.23848 secondi


### Rimozione del GIL

Esercizio 3 - incrementa somma

In [16]:
somma = 0  # definisco e inizializzo somma a 0

def incrementa_somma():
    global somma  # richiamo nel blocco corrente la variabile somma definita globalmente
    for _ in range(100_000_000):  # eseguo 1000 volte un ciclo che incrementa somma di 1
        somma += 1

In [20]:
t1 = threading.Thread(target=incrementa_somma)  # creo un thread che esegue la funzione incrementa_somma
t2 = threading.Thread(target=incrementa_somma)  # creo un secondo thread che esegue la funzione incrementa_somma

for i in [t1, t2]:  # creo un ciclo che itera su t1 e t2
    i.start()  # avvio i thread
for i in [t1, t2]:  # creo un ciclo che itera su t1 e t2
    i.join()  # aspetto che i thread terminino

print(f"La somma deve essere 200_000_000, il risultato ottenuto è: {somma}")

La somma deve essere 200_000_000, il risultato ottenuto è: 261124606
